In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 부산광역시, 합계, 연도, 남자, 여자
pd_data = pd.read_csv('성_및_연령별_1인가구__시군구_20230313213550.csv', encoding='cp949')
np_data = pd_data.to_numpy()
print(np_data)

[['부산광역시' '합계' 2015 164617 197132]
 ['부산광역시' '합계' 2016 168035 204377]
 ['부산광역시' '합계' 2017 176932 211967]
 ['부산광역시' '합계' 2018 183579 220829]
 ['부산광역시' '합계' 2019 191796 231431]
 ['부산광역시' '합계' 2020 206311 248896]
 ['부산광역시' '합계' 2021 222040 265322]]


1. np_data에서 '부산광역시', '합계' 제외, dtype을 np.int64로 지정

2. 2015-2021년까지 남자/여자 1인 가구 평균치

3. 2015-2021년까지 남자/여자 편차

4. 2015-2021년 남/녀 1인 가구 상관도  : 여자 1인 가구 수 증가 --> 남자 1인 가구 수 증가?

5. 2015-2021년 연도별 남/녀 1인 가구 비율   (ex 2015년 남자 164617/(165617+197132))

In [13]:
# np_data에서 '부산광역시', '합계' 제외, dtype을 np.int64로 지정

np_data = np.concatenate((np_data[:,:2],np_data[:,2:].astype(np.int64)),axis=1)
num_data = np_data[:,2:].astype(np.int64)
print(np_data)
print(num_data)

[['부산광역시' '합계' 2015 164617 197132]
 ['부산광역시' '합계' 2016 168035 204377]
 ['부산광역시' '합계' 2017 176932 211967]
 ['부산광역시' '합계' 2018 183579 220829]
 ['부산광역시' '합계' 2019 191796 231431]
 ['부산광역시' '합계' 2020 206311 248896]
 ['부산광역시' '합계' 2021 222040 265322]]
[[  2015 164617 197132]
 [  2016 168035 204377]
 [  2017 176932 211967]
 [  2018 183579 220829]
 [  2019 191796 231431]
 [  2020 206311 248896]
 [  2021 222040 265322]]


In [25]:
m_num = num_data[:,1]
print(m_num)
f_num = num_data[:,2]
print(f_num)


[164617 168035 176932 183579 191796 206311 222040]
[197132 204377 211967 220829 231431 248896 265322]
